# Random Forest

In [1]:
from src.trainning          import create_variables
from src.data_loader        import create_df
from src.evaluation         import show_confusion_matrix, show_metricas, show_feature_importances, show_auc_roc, show_especificidad, get_feature_importances
from src.trainning          import final_pipeline, apply_grid_search
from src.data_loader        import serialize_models
from src.parser             import YamlParser
from pathlib                import Path


YAML_FILE = Path.cwd() / "config.yaml"
yaml_parser = YamlParser()
config = yaml_parser.load_yaml(YAML_FILE)

df_model_rf     = create_df('raw_data')
model_pipeline  = final_pipeline(model_type=config['params']['model_type']['random_forest'])

In [2]:
X_train, X_test, y_train, y_test = create_variables(df_model_rf)

model           = apply_grid_search(model_pipeline)

model.fit(X_train, y_train)

rf_optimizado   = model.best_estimator_
rf_model        = rf_optimizado.named_steps['model']

pred            = rf_optimizado.predict(X_test)

In [3]:
metricas, accuracy_final    = show_metricas(y_test, pred)
especificidad               = show_especificidad(y_test, pred)

serialize_models(model, 'rf_01_model')

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.8377

Especificidad del Modelo Final Optimizado: 0.9009

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       222
           1       0.50      0.51      0.51        43

    accuracy                           0.84       265
   macro avg       0.70      0.71      0.70       265
weighted avg       0.84      0.84      0.84       265



**Accuracy**

A primera vista, un 0.84 es un valor muy bueno, pero en este contexto es engañoso. El dataset tiene un desbalanceo de clases muy grande. En los datos de test son 222 de la clase 0 y 43 de la clase 1. Es decir, si el modelo dijese, sin necesidad de entrenamiento, que 'Nadie se va de la empresa', ya estaria acertando un gran porcentaje de los casos, y acercandose asi al 82% de Accuracy. Por tanto, para dar por bueno esta valor, es necesario compararlo con las demás métricas.

**Recall**

Aqui el problema esta en la clase 1, los trabajadores que abandonan. El modelo es capaz de detectar unicamente el 51% de los empleados que se van. Más de la mitad de los abandonos pasan desapercibidos para el modelo.

**Precision**

Igual, la clase 0 no hay problema, pero la clase 1 es muy mala. De todos los trabajadores que el modelo marca como positivos, solo el 51% lo son.

**F1-Score**

Un 51% de F1-Score indica que el modelo no esta aprendiendo los patrones de la clase 1, pero con un 91%, si esta aprendiendo los patrones de la clase 0. Hay un equilibrio de recall y precision muy mediocre para la clase 1.

**Especificidad**

Acierta en un 90% los casos de VN(Verdaderos Negativos), es decir, que tiene un alto acierto en la gente que no se va de la empresa.

In [4]:
gr = show_confusion_matrix(y_test, pred, 'Random Forest')
gr

Verdaderos Negativos (TN): 199 (El modelo predijo correctamente que el empleado "Se queda").

Verdaderos Positivos (TP): 20 (El modelo predijo correctamente que el empleado "Se va").

Falsos Negativos (FN): 23 (El modelo dijo que el empleado "Se queda", pero en realidad se fue).

Falsos Positivos (FP): 23 (El modelo dijo que el empleado "Se va", pero en realidad se quedó).

En la matriz de confusion se observa claramente como el modelo acierta casi al completo los casos de trabajadores que no abandonan, pero falla estrepitosamente con los trabajadores que si lo hacen. Se esta pasando por alto a mas de la mitad de las bajas.

In [5]:
gr, auc_roc = show_auc_roc(model, X_test, y_test)
gr

El modelo ha conseguido un AUC de 0.74, un resultado aceptable. Esto se traduce en que hay un 75% de posibilidades de que, escogiendo aleatoriamente a un trabajador de la clase 0 y a otro de la clase 1, el modelo le asigne una mayor probabilidad de salida al segundo. Al principio, la curva tiene una subida muy pronunciada. Esto quiere decir que, las primeras bajas las detecta perfectamente. A partir de 0.2 en la tasa de falsos positivos, es donde ya empieza a tener más problemas, y clasifica verdaderos positivos cuando realmente no lo son, por eso la curva pasa a ir horizontalmente hacia la derecha.

In [6]:
feature_names = rf_optimizado.named_steps['preprocessing'].get_feature_names_out()

fi = get_feature_importances(rf_model, feature_names)
gr = show_feature_importances(fi)
gr

# Random Forest - SMOTEENN

In [7]:
model_pipeline                      = final_pipeline(model_type=config['params']['model_type']['random_forest'], use_smote=True)
X_train, X_test, y_train, y_test    = create_variables(df_model_rf)

model_sm    = apply_grid_search(model_pipeline)

model_sm.fit(X_train, y_train)

rf_optimizado   = model_sm.best_estimator_
rf_model        = rf_optimizado.named_steps['model']

pred = rf_optimizado.predict(X_test)

serialize_models(model_sm, 'rf_02_model_15v')

metricas, accuracy_final    = show_metricas(y_test, pred)
especificidad               = show_especificidad(y_test, pred)

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.7472

Especificidad del Modelo Final Optimizado: 0.7748

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.91      0.77      0.84       222
           1       0.34      0.60      0.44        43

    accuracy                           0.75       265
   macro avg       0.63      0.69      0.64       265
weighted avg       0.82      0.75      0.77       265



A estás métricas se le puede aplicar el mismo analisis del modelo anterior. Incluso estas son peores. Pensando que entrenar el modelo con menos ruido, iba a mejorar los resultados, ha pasado lo contrario. Random Forest suele apoyarse en todas las variables, aunque estas no sean muy importantes. Al entrenar el modelo con 15 variables, he eliminado señales, que por débiles que fueran, ayudaban al modelo a diferenciar mejor. En datasets pequeños y con datos que generan ruido, es mejor mantener todo el conjunto de datos.

In [8]:
gr = show_confusion_matrix(y_test, pred, 'Random Forest')
gr

In [9]:
gr, auc_roc = show_auc_roc(model_sm, X_test, y_test)
gr

El modelo ha conseguido un AUC de 0.72, que es bastante aceptable, aunque esta lejos de un modelo óptimo. Se puede observar como la subida no es tan agresiva como en el modelo anterior, pero si coincide en el mismo punto de Tasa de Falsos Positivos. Al pasar el 0.2, la curva se desplaza horizontalmente hacia la derecha, de forma más calmada. No se acerca demasiado a la esquina superior izquierda, lo que indica que el modelo no logra un equilibrio óptimo entre sensibilidad y especificidad en todos los umbrales. Aunque el modelo se aleja del azar(0.5), aun sigue lejos de un rendimiento óptimo.

In [10]:
feature_names = rf_optimizado.named_steps['preprocessing'].get_feature_names_out()

fi = get_feature_importances(rf_model, feature_names)
gr = show_feature_importances(fi)
gr

# Random Forest simple - Umbral Modificado

In [11]:
y_probs = model.predict_proba(X_test)[:, 1]

pred_optimo = ( y_probs >= 0.35 ).astype(int)

In [12]:
metricas, accuracy_final = show_metricas(y_test, pred_optimo)
especificidad = show_especificidad(y_test, pred_optimo)

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.6151

Especificidad del Modelo Final Optimizado: 0.5946

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.92      0.59      0.72       222
           1       0.26      0.72      0.38        43

    accuracy                           0.62       265
   macro avg       0.59      0.66      0.55       265
weighted avg       0.81      0.62      0.67       265



In [13]:
gr = show_confusion_matrix(y_test, pred_optimo, 'R.Forest - 15 Variables - Umbral Modificado')
gr